In [1]:
import pickle 
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.models import load_model

In [4]:
#### load the trained model, scaler, pickle, one hot
model = load_model('model.h5')

## load the encoder and scaler 
with open('geo_encoder.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [5]:
### EXAMPLE INPUT DATA
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age' : 40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000

}

In [6]:
input_df = pd.DataFrame([input_data])
label_encoder_geo = OneHotEncoder(sparse_output=False)

In [7]:
label_encoder_geo.fit(input_df[['Geography']])

OneHotEncoder(sparse_output=False)

In [8]:
## one-hot encode 'geography'
geo_encoded = label_encoder_geo.transform([input_df['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

o:\ANN\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France
0,1.0


In [9]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [15]:
#### encode categorical variables

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

ValueError: y contains previously unseen labels: 1

In [16]:
### concatination one hot encoded
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France
0,600,1,40,3,60000,2,1,1,50000,1.0


In [19]:
missing_cols = [col for col in scaler.feature_names_in_ if col not in input_df.columns]
for col in missing_cols:
    input_df[col] = 0

In [20]:
combined_df = input_df[scaler.feature_names_in_]

In [21]:
input_scaled = scaler.transform(input_df)

In [22]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [24]:
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 31ms/step


array([[0.04252798]], dtype=float32)

In [25]:
prediction_prob = prediction[0][0]

In [26]:
prediction_prob

0.04252798